In [2]:
# imports
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
import rpsalweaklydet as rps

In [3]:
path = Path('/home/renato/data/chestxray8')

# Train/Validation  splits

In [4]:
images_path = path/'images'
len(images_path.ls())

112120

In [5]:
df = pd.read_csv(path/'Data_Entry_2017.csv')
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [6]:
df_bbx = pd.read_csv(path/'BBox_List_2017.csv')
df_bbx.head()

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,NaN,NaN,NaN
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,NaN,NaN,NaN
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,NaN,NaN,NaN
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,NaN,NaN,NaN
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,NaN,NaN,NaN


In [7]:
train_val_files = pd.read_csv(path/'train_val_list.txt', header=None)[0].tolist()
test_files = pd.read_csv(path/'test_list.txt', header=None)[0].tolist()
len(train_val_files),len(test_files)

(86524, 25596)

In [11]:
train_val_df = df[df['Image Index'].isin(train_val_files)]
train_val_df.shape

(86524, 12)

In [29]:
np.random.seed(34)
idxs = np.random.permutation(train_val_df['Patient ID'].unique())

In [30]:
valid_files = train_val_df[train_val_df['Patient ID'].isin(idxs[:3551])]['Image Index']
train_files = train_val_df[~train_val_df['Patient ID'].isin(idxs[:3551])]['Image Index']
train_files.to_csv(path/'train_list.txt', index=False, header=False)
valid_files.to_csv(path/'valid_list.txt', index=False, header=False)
train_files.shape,valid_files.shape

((75300,), (11224,))

In [51]:
train_files = train_files.tolist()
valid_files = valid_files.tolist()

len(train_files),len(valid_files),len(test_files)

(75300, 11224, 25596)

# Build final DF

In [49]:
df_bbx['Finding Label'].replace('Infiltrate', 'Infiltration', inplace=True)

In [55]:
df_final = df.iloc[:,[0,1,]].copy()
df_final.columns = ['file','label']
df_final['set'] = 'Train'
df_final['set'].loc[df_final.file.isin(valid_files)] = 'Valid'
df_final['set'].loc[df_final.file.isin(test_files)] = 'Test'
df_final.head()

,file,label,set
0,00000001_000.png,Cardiomegaly,Train
1,00000001_001.png,Cardiomegaly|Emphysema,Train
2,00000001_002.png,Cardiomegaly|Effusion,Train
3,00000002_000.png,No Finding,Train
4,00000003_000.png,Hernia,Test


In [59]:
df_final['label'] = df_final['label'].str.split('|')
df_final = df_final.explode('label')
labels = sorted(o for o in df_final['label'].explode().unique() if o != 'No Finding')
lbl2idx = {k:i+1 for i,k in enumerate(labels)}
lbl2idx['No Finding'] = 0
print(lbl2idx)

{'Atelectasis': 1, 'Cardiomegaly': 2, 'Consolidation': 3, 'Edema': 4, 'Effusion': 5, 'Emphysema': 6, 'Fibrosis': 7, 'Hernia': 8, 'Infiltration': 9, 'Mass': 10, 'Nodule': 11, 'Pleural_Thickening': 12, 'Pneumonia': 13, 'Pneumothorax': 14, 'No Finding': 0}


In [60]:
df_bbx_final = df_bbx.iloc[:,:6].copy()
df_bbx_final.columns = ['file', 'label', 'x' ,'y', 'w', 'h']
df_bbx_final['top'] = df_bbx_final.y
df_bbx_final['left'] = df_bbx_final.x
df_bbx_final['bottom'] = df_bbx_final.y + df_bbx_final.h
df_bbx_final['right'] = df_bbx_final.x + df_bbx_final.w
df_bbx_final.drop(['x','y','w','h'], axis=1, inplace=True)
df_final = df_final.merge(df_bbx_final, 'left', on=['file','label'])
df_final['bbx'] = True
df_final.loc[df_final['top'].isna(), 'bbx'] = False
df_final['top'].fillna(0, inplace=True)
df_final['left'].fillna(0, inplace=True)
df_final['bottom'].fillna(1024, inplace=True)
df_final['right'].fillna(1024, inplace=True)
df_final.head()

,file,label,set,top,left,bottom,right,bbx
0,00000001_000.png,Cardiomegaly,Train,0.0,0.0,1024.0,1024.0,False
1,00000001_001.png,Cardiomegaly,Train,0.0,0.0,1024.0,1024.0,False
2,00000001_001.png,Emphysema,Train,0.0,0.0,1024.0,1024.0,False
3,00000001_002.png,Cardiomegaly,Train,0.0,0.0,1024.0,1024.0,False
4,00000001_002.png,Effusion,Train,0.0,0.0,1024.0,1024.0,False


In [61]:
def process_labels(x):
    lbls = x.label.tolist()
    bbx = np.array(x[['top', 'left', 'bottom', 'right']]).tolist()
    return [bbx,lbls]

In [63]:
%time df_full_bbx = df_final.groupby(['file','set']).apply(process_labels).reset_index(name='label')

CPU times: user 55.3 s, sys: 127 ms, total: 55.5 s
Wall time: 55.5 s


In [66]:
df_full_bbx['bbx'] = df_full_bbx.file.isin(df_bbx['Image Index'])
df_full_bbx.head()

,file,set,label,bbx
0,00000001_000.png,Train,"[[[0.0, 0.0, 1024.0, 1024.0]], [Cardiomegaly]]",False
1,00000001_001.png,Train,"[[[0.0, 0.0, 1024.0, 1024.0], [0.0, 0.0, 1024....",False
2,00000001_002.png,Train,"[[[0.0, 0.0, 1024.0, 1024.0], [0.0, 0.0, 1024....",False
3,00000002_000.png,Train,"[[[0.0, 0.0, 1024.0, 1024.0]], [No Finding]]",False
4,00000003_000.png,Test,"[[[0.0, 0.0, 1024.0, 1024.0]], [Hernia]]",False


In [67]:
df_full_bbx.query("file=='00000032_037.png'").label.iloc[0]

[[[0.0, 0.0, 1024.0, 1024.0],
  [0.0, 0.0, 1024.0, 1024.0],
  [119.195767195767, 339.166137566138, 470.2814814814811, 511.458201058202]],
 ['Cardiomegaly', 'Edema', 'Infiltration']]

In [68]:
df_full_bbx.to_pickle(path/'full_ds_bbx.pkl')

In [69]:
df_full_bbx = pd.read_pickle(path/'full_ds_bbx.pkl')
df_full_bbx.tail()

,file,set,label,bbx
112115,00030801_001.png,Train,"[[[0.0, 0.0, 1024.0, 1024.0], [0.0, 0.0, 1024....",False
112116,00030802_000.png,Test,"[[[0.0, 0.0, 1024.0, 1024.0]], [No Finding]]",False
112117,00030803_000.png,Test,"[[[0.0, 0.0, 1024.0, 1024.0]], [No Finding]]",False
112118,00030804_000.png,Test,"[[[0.0, 0.0, 1024.0, 1024.0]], [No Finding]]",False
112119,00030805_000.png,Test,"[[[0.0, 0.0, 1024.0, 1024.0]], [No Finding]]",False
